<a href="https://colab.research.google.com/github/DENGCPU/PyTorch-Study/blob/main/PyTorch%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 定义神经网络
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [3]:
# MNIST数据集期望输入维度是32X32 , 所以需要将输入的图片维度转换
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0657,  0.0405, -0.1459, -0.0732, -0.0718,  0.0422,  0.0698, -0.0418,
          0.0379,  0.0775]], grad_fn=<AddmmBackward0>)


In [4]:
# 将所有参数缓存器置零，用随机的梯度来方向传播
net.zero_grad()
out.backward(torch.randn(1,10))

# 计算损失
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()  # 均方误差
loss = criterion(output,target)
print(loss)

tensor(1.4830, grad_fn=<MseLossBackward0>)


In [ ]:
# 反向传播步骤
# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
#       -> view -> linear -> relu -> linear -> relu -> linear
#       -> MSELoss
#       -> loss

In [5]:
print(loss.grad_fn)

In [6]:
print(loss.grad_fn.next_functions[0][0])

In [7]:
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [8]:
# 将现存的梯度清零
net.zero_grad()
print('conv1.bias.grad before backward:', net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward:', net.conv1.bias.grad)

conv1.bias.grad before backward: None
conv1.bias.grad after backward: tensor([ 0.0045,  0.0005,  0.0037,  0.0085,  0.0166, -0.0021])


In [9]:
# 更新神经网络参数
# weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [10]:
# 或者使用torch.optim包提供的更新方法
import torch.optim as optim
# 创建optimizer
optimizer = optim.SGD(net.parameters(),lr=0.01)

# 在训练loop中
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()  #执行更新